In [5]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import glob
%matplotlib inline

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [6]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [205]:
tracklist = sorted(glob.glob(path+'tracks_nio/*2011*'))

In [206]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20111029.csv']

In [207]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2011.nc',datapath+'/rhum/rhum_2011.nc'])

In [208]:
#ssingle level variables
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm1.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [209]:
###
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCs_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outu10','outv10','out2mdewtmp','out2mtmp','outconv_ppt','outtot_cld_ice',\
           'outtot_cldwtr','outtot_cld_rain','outvi_div_cld_froz_wtr','outvi_div_cld_liq_wtr','outvi_div_gpot_flux',\
           'outvi_div_ke_flux','outvi_div_mass_flux','outvi_div_moisture_flux','outvi_div_olr_flux','outvi_div_tot_enrgy_flux',\
           'outvi_ke','outvi_pe_inte','outvi_pe_ie_latentenrgy','outvi_temp','outvi_olr','outvi_tot_enrgy','outvi_moisture_div']
   
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCs_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [210]:
###radiation variables
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outcape','outinst_10m_wnd_gst','outinst_moisture_flux','outinst_ssh_flux','outsurfmean_swr_flux','outsurfmean_lhf',\
           'outsurfmean_lwr_flux','outsurfmean_shf','outdwnwrdmean_swr_flux','outtopmean_lwr_flux','outtopmean_swr_flux',\
           'outvimean_moisture_div','outsurf_lhf','outsurf_shf','outtot_suprcool_liqwtr','outtot_wtr_vpr']
    
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [211]:
#########precip, mslp, sst variables############
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2011.nc',datapath+'/mslp/mslp_2011.nc',datapath+'/sst/sst_2011.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2011)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outmslp','outconv_rrate','outls_rrate','outmn_conv_prate',\
                 'outmn_ls_prate','outmn_tot_prate','outsst']
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [212]:
#Pressure vars - vor hum
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvort','outrhum']
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [213]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2011.nc')

In [214]:
# Pressure vars - geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outgpot']
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [215]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2011.nc')

In [216]:
#Pressure vars - vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvvel']
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [217]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [218]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2011.nc')

In [219]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [220]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20111029.csv']

In [221]:
#storms 1
mainkeila=pd.read_csv(output+'inner/inner_nio_all_keila.csv',delimiter=r",")
mainkeila.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainkeila=mainkeila.drop('a', axis=1)

pdf_keila=pd.read_csv(target+'2011_pmin_nio_keila.csv',delimiter=r",")
pdf_keila.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_keila=pdf_keila.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2011_nio_keila.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_keila=pd.DataFrame.from_dict(TCs_ts[0])
rlev_keila=pd.DataFrame.from_dict(TCr_ts[0])
prlev_keila=pd.DataFrame.from_dict(TCpr_ts[0])
prlev_keila['outmslp']=prlev_keila['outmslp']/100

ts1_keila=pd.concat([mainkeila,extra,slev_keila,rlev_keila,prlev_keila], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_keila=pd.DataFrame.from_dict(alldict1)

ts_keila=pd.concat([pdf_keila,ts1_keila,plev_keila], axis=1, join='inner')
ts_keila

Pmin     w10max        div100        div200        div250  \
0   1004.162500  12.055099  2.367005e-06  5.928226e-06 -6.653256e-06   
1   1001.153125  11.664788  5.151447e-06  6.038431e-06 -5.926158e-06   
2   1000.553750  12.076045  4.491908e-06 -2.377669e-06 -8.104904e-06   
3   1001.921250  11.888578 -8.037207e-07  3.972694e-06 -3.771493e-06   
4   1002.775000  12.550705 -4.033099e-06  1.094998e-05 -5.173708e-06   
5   1001.188750  12.083922 -2.131600e-06  3.880810e-06 -8.327203e-06   
6   1000.329375  12.904962 -2.631009e-06 -4.133747e-06 -1.010388e-05   
7   1001.649375  12.650217  1.991523e-05 -4.412064e-06 -1.392189e-05   
8   1002.839375  13.250352  3.119516e-05 -4.678834e-06 -2.291874e-05   
9   1000.342500  13.007581  2.157145e-05 -2.252393e-06 -1.759384e-05   
10   999.466875  13.175367  1.000858e-05  2.954720e-06 -8.973057e-06   
11  1001.390625  14.137432  7.083080e-06  2.711922e-06 -5.611508e-06   
12  1001.234375  13.275363  1.647588e-05 -6.240636e-06 -1.009992e-05   
13   999.846875  13.511723  1.056031e-05 -8.567108e-06 -1.858777e-05   
14   999.858750  13.525671  2.137747e-05 -3.657515e-06 -1.718478e-05   
15  1000.438750  12.950952  2.699003e-05 -7.016030e-06 -1.261260e-05   
16  1001.784375  13.652727  4.422815e-05  2.403215e-05  4.384571e-06   
17   999.697500  13.646322  5.644867e-05  1.861957e-05 -3.470981e-06   
18   999.407500  14.536643  3.529304e-05 -6.994348e-06 -1.127301e-05   
19  1000.673125  16.357458  4.446614e-05  9.985470e-06 -8.682682e-06   
20  1001.375625  16.069832  5.945713e-05  2.021193e-05 -1.064240e-06   
21   999.451250  15.783942  3.680948e-05  1.035104e-05 -6.856630e-06   
22   997.816875  15.197276  2.018136e-05  2.724823e-07 -1.755262e-05   
23   997.781250  14.846285  2.439742e-05 -1.100823e-05 -1.889204e-05   
24   998.343125  15.889249  1.592160e-05 -6.337809e-06 -1.895754e-05   
25   996.635625  17.598475  4.526514e-05  2.442158e-05  8.147922e-06   
26   996.980625  17.598507  5.284435e-05  2.803545e-05  1.653575e-05   
27   998.413750  19.337402  5.982551e-05  5.110669e-05  2.604697e-05   
28   998.430000  19.128256  6.385486e-05  5.033510e-05  3.036293e-06   
29   996.298125  18.455315  4.831482e-05  3.642469e-05 -1.118409e-05   
30   995.861875  17.359365  2.655158e-05  3.087919e-05  6.509329e-07   
31   997.030625  15.830731  3.155840e-05  3.737467e-05 -2.937823e-06   
32   998.573750  14.125549  3.397941e-05  3.196812e-05 -1.792073e-05   
33   996.673750  14.234856  4.604636e-05  2.395063e-05 -2.470041e-05   
34   998.388125  12.302150  6.834936e-05  3.212259e-05 -1.573827e-05   
35  1000.567500  13.596095  7.703467e-05  4.542133e-05 -5.477450e-06   
36  1003.009375  14.185441  5.224041e-05  2.572789e-05 -9.827527e-06   
37  1002.793125  13.296937  5.141958e-05  1.864140e-05 -7.762996e-06   
38  1000.680000  12.367274  3.323522e-05  2.750498e-05 -1.203175e-05   
39  1001.320625  11.381276  1.082632e-05  1.873321e-05 -9.771768e-06   
40  1003.835625  10.595592 -2.277343e-06  1.735205e-05 -1.013863e-05   
41  1002.569375  10.759361 -2.510142e-06  1.253501e-05 -8.929334e-06   
42  1003.453750  11.476065  7.590203e-06 -9.532623e-07 -8.952811e-06   
43  1005.141250  11.670926  7.127824e-06  6.485893e-06 -9.388616e-06   
44  1006.389375  11.541632  1.195189e-06  2.151980e-05 -5.252372e-06   
45  1005.676875   9.210520  1.220107e-06  3.394261e-05  3.550867e-06   
46  1005.288750   8.805575 -9.278854e-06  5.321188e-05 -2.335201e-06   
47  1007.045625   7.341660 -1.702751e-05  3.424481e-05 -1.219734e-05   
48  1008.978125   7.700410 -5.553022e-06  1.100660e-05 -1.490095e-05   
49  1007.689375   8.404633  4.035020e-06  3.817001e-06 -9.082682e-06   
50  1006.245000   8.886277  7.853602e-06  1.690291e-05  1.789233e-07   
51  1007.603125   9.780406  5.205176e-06  1.469755e-05  4.356302e-06   
52  1009.020625   8.787664  5.732990e-07  1.885274e-05  8.316190e-06   
53  1008.416250   7.375129  5.940783e-06  1.837788e-05  2.634721e-06   
54  1008.088125   7.326790  1.749904e-05  1.589581e

In [222]:
df1=ts_keila
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

Pmin     w10max        div100        div200        div250  \
0   1004.162500  12.055099  2.367005e-06  5.928226e-06 -6.653256e-06   
1   1001.153125  11.664788  5.151447e-06  6.038431e-06 -5.926158e-06   
2   1000.553750  12.076045  4.491908e-06 -2.377669e-06 -8.104904e-06   
3   1001.921250  11.888578 -8.037207e-07  3.972694e-06 -3.771493e-06   
4   1002.775000  12.550705 -4.033099e-06  1.094998e-05 -5.173708e-06   
5   1001.188750  12.083922 -2.131600e-06  3.880810e-06 -8.327203e-06   
6   1000.329375  12.904962 -2.631009e-06 -4.133747e-06 -1.010388e-05   
7   1001.649375  12.650217  1.991523e-05 -4.412064e-06 -1.392189e-05   
8   1002.839375  13.250352  3.119516e-05 -4.678834e-06 -2.291874e-05   
9   1000.342500  13.007581  2.157145e-05 -2.252393e-06 -1.759384e-05   
10   999.466875  13.175367  1.000858e-05  2.954720e-06 -8.973057e-06   
11  1001.390625  14.137432  7.083080e-06  2.711922e-06 -5.611508e-06   
12  1001.234375  13.275363  1.647588e-05 -6.240636e-06 -1.009992e-05   
13   999.846875  13.511723  1.056031e-05 -8.567108e-06 -1.858777e-05   
14   999.858750  13.525671  2.137747e-05 -3.657515e-06 -1.718478e-05   
15  1000.438750  12.950952  2.699003e-05 -7.016030e-06 -1.261260e-05   
16  1001.784375  13.652727  4.422815e-05  2.403215e-05  4.384571e-06   
17   999.697500  13.646322  5.644867e-05  1.861957e-05 -3.470981e-06   
18   999.407500  14.536643  3.529304e-05 -6.994348e-06 -1.127301e-05   
19  1000.673125  16.357458  4.446614e-05  9.985470e-06 -8.682682e-06   
20  1001.375625  16.069832  5.945713e-05  2.021193e-05 -1.064240e-06   
21   999.451250  15.783942  3.680948e-05  1.035104e-05 -6.856630e-06   
22   997.816875  15.197276  2.018136e-05  2.724823e-07 -1.755262e-05   
23   997.781250  14.846285  2.439742e-05 -1.100823e-05 -1.889204e-05   
24   998.343125  15.889249  1.592160e-05 -6.337809e-06 -1.895754e-05   
25   996.635625  17.598475  4.526514e-05  2.442158e-05  8.147922e-06   
26   996.980625  17.598507  5.284435e-05  2.803545e-05  1.653575e-05   
27   998.413750  19.337402  5.982551e-05  5.110669e-05  2.604697e-05   
28   998.430000  19.128256  6.385486e-05  5.033510e-05  3.036293e-06   
29   996.298125  18.455315  4.831482e-05  3.642469e-05 -1.118409e-05   
30   995.861875  17.359365  2.655158e-05  3.087919e-05  6.509329e-07   
31   997.030625  15.830731  3.155840e-05  3.737467e-05 -2.937823e-06   
32   998.573750  14.125549  3.397941e-05  3.196812e-05 -1.792073e-05   
33   996.673750  14.234856  4.604636e-05  2.395063e-05 -2.470041e-05   
34   998.388125  12.302150  6.834936e-05  3.212259e-05 -1.573827e-05   
35  1000.567500  13.596095  7.703467e-05  4.542133e-05 -5.477450e-06   
36  1003.009375  14.185441  5.224041e-05  2.572789e-05 -9.827527e-06   
37  1002.793125  13.296937  5.141958e-05  1.864140e-05 -7.762996e-06   
38  1000.680000  12.367274  3.323522e-05  2.750498e-05 -1.203175e-05   
39  1001.320625  11.381276  1.082632e-05  1.873321e-05 -9.771768e-06   
40  1003.835625  10.595592 -2.277343e-06  1.735205e-05 -1.013863e-05   
41  1002.569375  10.759361 -2.510142e-06  1.253501e-05 -8.929334e-06   
42  1003.453750  11.476065  7.590203e-06 -9.532623e-07 -8.952811e-06   
43  1005.141250  11.670926  7.127824e-06  6.485893e-06 -9.388616e-06   
44  1006.389375  11.541632  1.195189e-06  2.151980e-05 -5.252372e-06   
45  1005.676875   9.210520  1.220107e-06  3.394261e-05  3.550867e-06   
46  1005.288750   8.805575 -9.278854e-06  5.321188e-05 -2.335201e-06   
47  1007.045625   7.341660 -1.702751e-05  3.424481e-05 -1.219734e-05   
48  1008.978125   7.700410 -5.553022e-06  1.100660e-05 -1.490095e-05   
49  1007.689375   8.404633  4.035020e-06  3.817001e-06 -9.082682e-06   
50  1006.245000   8.886277  7.853602e-06  1.690291e-05  1.789233e-07   
51  1007.603125   9.780406  5.205176e-06  1.469755e-05  4.356302e-06   
52  1009.020625   8.787664  5.732990e-07  1.885274e-05  8.316190e-06   
53  1008.416250   7.375129  5.940783e-06  1.837788e-05  2.634721e-06   
54  1008.088125   7.326790  1.749904e-05  1.589581e

In [223]:
df1.to_csv(output+'timeseries_all_nio_keila.csv')

In [196]:
mainphailin=pd.read_csv(output+'inner/inner_nio_all_phailin.csv',delimiter=r",")
mainphailin.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainphailin=mainphailin.drop('a', axis=1)

pdf_phailin=pd.read_csv(target+'2013_pmin_nio_phailin.csv',delimiter=r",")
pdf_phailin.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_phailin=pdf_phailin.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2013_nio_phailin.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_phailin=pd.DataFrame.from_dict(TCs_ts[1])
rlev_phailin=pd.DataFrame.from_dict(TCr_ts[1])
prlev_phailin=pd.DataFrame.from_dict(TCpr_ts[1])
prlev_phailin['outmslp']=prlev_phailin['outmslp']/100

ts1_phailin=pd.concat([mainphailin,extra,slev_phailin,rlev_phailin,prlev_phailin], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[1]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[1]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[1]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[1]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_phailin=pd.DataFrame.from_dict(alldict1)

ts_phailin=pd.concat([pdf_phailin,ts1_phailin,plev_phailin], axis=1, join='inner')
ts_phailin

Pmin     w10max    div100    div200        div250        div300  \
0   1004.541250  12.276583  0.000029  0.000048  1.633112e-05  2.857541e-07   
1   1005.962500  12.332129  0.000036  0.000053  2.187599e-05 -8.693290e-07   
2   1004.043125  13.427410  0.000043  0.000052  1.999416e-05  6.500800e-06   
3   1001.621875  13.364658  0.000060  0.000057  2.090617e-05  1.322057e-05   
4   1002.404375  14.744117  0.000080  0.000047  1.813365e-05  1.923292e-05   
5   1003.551250  15.033587  0.000073  0.000052  1.498547e-05  1.701436e-05   
6   1000.575000  16.546116  0.000085  0.000041  1.024381e-05  6.281029e-06   
7    998.475625  17.705784  0.000079  0.000024 -2.110542e-06 -2.017151e-06   
8    999.773125  18.141623  0.000060  0.000019  7.846455e-06 -4.622073e-06   
9   1001.150625  17.423697  0.000055  0.000015  1.350930e-05  2.925502e-06   
10  1000.231875  15.812882  0.000068  0.000020  8.399742e-06  4.318872e-07   
11   998.685625  15.824848  0.000067  0.000028  1.138419e-06 -3.944097e-07   
12   998.988125  17.211996  0.000054  0.000022  3.823806e-06  5.108910e-06   
13  1001.071250  17.943642  0.000044  0.000020  1.132647e-05  9.094231e-06   
14  1000.124375  16.857670  0.000063  0.000016  5.804088e-07 -1.204985e-06   
15   998.048750  18.347443  0.000062  0.000018  2.267064e-06  7.912022e-06   
16   997.688750  17.747591  0.000063  0.000024  1.159571e-05  1.100497e-05   
17   999.379375  16.936386  0.000051  0.000023  6.386067e-06  3.068328e-06   
18   998.268125  17.027782  0.000052  0.000039  6.391296e-06  5.094574e-06   
19   996.313125  17.837368  0.000065  0.000044  1.073909e-05 -4.946587e-06   
20   997.167500  17.500372  0.000063  0.000046  2.002710e-05 -1.017483e-06   
21   998.083750  17.212761  0.000035  0.000029  1.897555e-05  1.348670e-06   
22   996.211875  17.693806  0.000042  0.000014  1.002951e-05  5.811314e-06   
23   993.668125  17.948174  0.000032  0.000019  5.530843e-06  1.652733e-06   
24   993.532500  18.354475  0.000026  0.000034  2.540330e-05  1.716960e-05   
25   994.379375  19.312447  0.000050  0.000060  2.570291e-05  1.001679e-05   
26   992.683750  18.928219  0.000052  0.000079  2.509523e-05  7.362317e-06   
27   989.864375  19.780880  0.000081  0.000073  5.215813e-05  1.839990e-05   
28   990.645625  18.425493  0.000055  0.000075  6.633886e-05  1.610217e-05   
29   991.225625  19.711962  0.000094  0.000071  3.840161e-05 -3.456176e-06   
30   988.725000  19.743273  0.000082  0.000051  1.461341e-05 -1.304228e-05   
31   985.784375  19.935196  0.000060  0.000030  4.462594e-06 -5.760219e-06   
32   984.538750  21.063271  0.000050  0.000031  4.779259e-06 -2.888521e-07   
33   984.752500  23.133354  0.000060  0.000039  1.936996e-05  1.750640e-07   
34   982.734375  21.464325  0.000077  0.000048  2.585245e-05  1.047139e-05   
35   979.214375  23.078482  0.000103  0.000061  2.502955e-05  3.712189e-06   
36   976.015000  25.583275  0.000093  0.000071  4.319590e-05  6.113768e-06   
37   975.251875  25.520428  0.000104  0.000048  3.682808e-05  1.004808e-05   
38   972.612500  27.095358  0.000075  0.000039  3.338525e-05  1.252212e-05   
39   968.713125  27.809877  0.000061  0.000037  3.739761e-05  2.713503e-05   
40   970.998750  27.449768  0.000046  0.000055  5.931163e-05  3.101548e-05   
41   975.134375  26.666496  0.000035  0.000045  4.113558e-05  2.403348e-05   
42   979.125625  26.435585  0.000062  0.000045  1.512366e-05  1.503695e-05   

          div400        div500     eqt1000      eqt200  ...  outvvel_250  \
0  -4.733704e-06 -2.174062e-06  333.454468  350.030762  ...    -0.044317   
1  -5.573118e-06 -3.450627e-06  333.575043  349.363861  ...    -0.045163   
2  -1.089153e-06 -6.368437e-07  333.535156  348.932495  ...    -0.056222   
3  -7.693538e-06 -3.845886e-06  333.355042  348.859222  ...    -0.060638   
4  -1.534938e-05 -1.006709e-05  333.566162  350.008545  ...    -0.050325   
5  -1.805706e-05 -2.294027e-06  332.904816  350.923584  ...    -0.054135   
6  -1.362825e-05  9.389391e-07  333.497345  35

In [197]:
df1=ts_phailin
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

Pmin     w10max    div100    div200        div250        div300  \
0   1004.541250  12.276583  0.000029  0.000048  1.633112e-05  2.857541e-07   
1   1005.962500  12.332129  0.000036  0.000053  2.187599e-05 -8.693290e-07   
2   1004.043125  13.427410  0.000043  0.000052  1.999416e-05  6.500800e-06   
3   1001.621875  13.364658  0.000060  0.000057  2.090617e-05  1.322057e-05   
4   1002.404375  14.744117  0.000080  0.000047  1.813365e-05  1.923292e-05   
5   1003.551250  15.033587  0.000073  0.000052  1.498547e-05  1.701436e-05   
6   1000.575000  16.546116  0.000085  0.000041  1.024381e-05  6.281029e-06   
7    998.475625  17.705784  0.000079  0.000024 -2.110542e-06 -2.017151e-06   
8    999.773125  18.141623  0.000060  0.000019  7.846455e-06 -4.622073e-06   
9   1001.150625  17.423697  0.000055  0.000015  1.350930e-05  2.925502e-06   
10  1000.231875  15.812882  0.000068  0.000020  8.399742e-06  4.318872e-07   
11   998.685625  15.824848  0.000067  0.000028  1.138419e-06 -3.944097e-07   
12   998.988125  17.211996  0.000054  0.000022  3.823806e-06  5.108910e-06   
13  1001.071250  17.943642  0.000044  0.000020  1.132647e-05  9.094231e-06   
14  1000.124375  16.857670  0.000063  0.000016  5.804088e-07 -1.204985e-06   
15   998.048750  18.347443  0.000062  0.000018  2.267064e-06  7.912022e-06   
16   997.688750  17.747591  0.000063  0.000024  1.159571e-05  1.100497e-05   
17   999.379375  16.936386  0.000051  0.000023  6.386067e-06  3.068328e-06   
18   998.268125  17.027782  0.000052  0.000039  6.391296e-06  5.094574e-06   
19   996.313125  17.837368  0.000065  0.000044  1.073909e-05 -4.946587e-06   
20   997.167500  17.500372  0.000063  0.000046  2.002710e-05 -1.017483e-06   
21   998.083750  17.212761  0.000035  0.000029  1.897555e-05  1.348670e-06   
22   996.211875  17.693806  0.000042  0.000014  1.002951e-05  5.811314e-06   
23   993.668125  17.948174  0.000032  0.000019  5.530843e-06  1.652733e-06   
24   993.532500  18.354475  0.000026  0.000034  2.540330e-05  1.716960e-05   
25   994.379375  19.312447  0.000050  0.000060  2.570291e-05  1.001679e-05   
26   992.683750  18.928219  0.000052  0.000079  2.509523e-05  7.362317e-06   
27   989.864375  19.780880  0.000081  0.000073  5.215813e-05  1.839990e-05   
28   990.645625  18.425493  0.000055  0.000075  6.633886e-05  1.610217e-05   
29   991.225625  19.711962  0.000094  0.000071  3.840161e-05 -3.456176e-06   
30   988.725000  19.743273  0.000082  0.000051  1.461341e-05 -1.304228e-05   
31   985.784375  19.935196  0.000060  0.000030  4.462594e-06 -5.760219e-06   
32   984.538750  21.063271  0.000050  0.000031  4.779259e-06 -2.888521e-07   
33   984.752500  23.133354  0.000060  0.000039  1.936996e-05  1.750640e-07   
34   982.734375  21.464325  0.000077  0.000048  2.585245e-05  1.047139e-05   
35   979.214375  23.078482  0.000103  0.000061  2.502955e-05  3.712189e-06   
36   976.015000  25.583275  0.000093  0.000071  4.319590e-05  6.113768e-06   
37   975.251875  25.520428  0.000104  0.000048  3.682808e-05  1.004808e-05   
38   972.612500  27.095358  0.000075  0.000039  3.338525e-05  1.252212e-05   
39   968.713125  27.809877  0.000061  0.000037  3.739761e-05  2.713503e-05   
40   970.998750  27.449768  0.000046  0.000055  5.931163e-05  3.101548e-05   
41   975.134375  26.666496  0.000035  0.000045  4.113558e-05  2.403348e-05   
42   979.125625  26.435585  0.000062  0.000045  1.512366e-05  1.503695e-05   

          div400        div500     eqt1000      eqt200  ...  outvvel_250  \
0  -4.733704e-06 -2.174062e-06  333.454468  350.030762  ...    -0.044317   
1  -5.573118e-06 -3.450627e-06  333.575043  349.363861  ...    -0.045163   
2  -1.089153e-06 -6.368437e-07  333.535156  348.932495  ...    -0.056222   
3  -7.693538e-06 -3.845886e-06  333.355042  348.859222  ...    -0.060638   
4  -1.534938e-05 -1.006709e-05  333.566162  350.008545  ...    -0.050325   
5  -1.805706e-05 -2.294027e-06  332.904816  350.923584  ...    -0.054135   
6  -1.362825e-05  9.389391e-07  333.497345  35

In [198]:
df1.to_csv(output+'timeseries_all_nio_phailin.csv')

In [199]:
mainlehar=pd.read_csv(output+'inner/inner_nio_all_lehar.csv',delimiter=r",")
mainlehar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainlehar=mainlehar.drop('a', axis=1)

pdf_lehar=pd.read_csv(target+'2013_pmin_nio_lehar.csv',delimiter=r",")
pdf_lehar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_lehar=pdf_lehar.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2013_nio_lehar.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_lehar=pd.DataFrame.from_dict(TCs_ts[2])
rlev_lehar=pd.DataFrame.from_dict(TCr_ts[2])
prlev_lehar=pd.DataFrame.from_dict(TCpr_ts[2])
prlev_lehar['outmslp']=prlev_lehar['outmslp']/100

ts1_lehar=pd.concat([mainlehar,extra,slev_lehar,rlev_lehar,prlev_lehar], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[2]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[2]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[2]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[2]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_lehar=pd.DataFrame.from_dict(alldict1)

ts_lehar=pd.concat([pdf_lehar,ts1_lehar,plev_lehar], axis=1, join='inner')
ts_lehar

Pmin     w10max        div100    div200        div250  \
0   1008.354375  12.089149 -1.114373e-06  0.000028  4.712652e-05   
1   1006.899375  12.099492  2.207192e-05  0.000028  4.880802e-05   
2   1008.137500  12.946914  3.799148e-05  0.000048  5.157499e-05   
3   1009.246250  13.524454  6.559324e-05  0.000044  4.015736e-05   
4   1007.101250  11.577554  6.630583e-05  0.000039  3.377555e-05   
..          ...        ...           ...       ...           ...   
64  1004.356875  15.110038  2.514235e-05 -0.000003 -1.469511e-05   
65  1003.528125  13.975189  1.180500e-05 -0.000001 -8.951417e-06   
66  1004.117500  13.896652 -1.453482e-07  0.000016  9.460861e-06   
67  1006.631250  13.558363  4.308552e-06  0.000013  6.167821e-06   
68  1006.995000  13.248140  1.028076e-05  0.000005 -6.598098e-07   

          div300        div400        div500     eqt1000      eqt200  ...  \
0   2.683366e-05  5.358663e-06 -3.709186e-06  321.301117  347.093323  ...   
1   3.506813e-05  3.678299e-06 -7.396683e-06  328.678925  347.569061  ...   
2   4.190413e-05 -3.900933e-06 -1.667449e-07  330.235901  349.061005  ...   
3   2.956255e-05 -7.985167e-06  9.848300e-07  330.619781  349.082031  ...   
4   1.697406e-05 -1.857898e-07 -9.271409e-07  329.841675  349.316895  ...   
..           ...           ...           ...         ...         ...  ...   
64 -3.176734e-06  1.943779e-06  8.115904e-09  328.671539  348.846008  ...   
65  1.838834e-06  6.610712e-06  1.255684e-06  327.732666  348.234741  ...   
66  3.836846e-06  5.894540e-07 -8.441037e-06  322.099609  348.801941  ...   
67  4.318058e-06  4.497268e-06 -5.560843e-06  322.856384  348.744019  ...   
68  1.322514e-07  6.795401e-06  2.703427e-06  327.365234  348.565765  ...   

    outvvel_250  outvvel_300  outvvel_400  outvvel_500  outvvel_600  \
0     -0.075850    -0.080951    -0.104787    -0.111518    -0.102335   
1     -0.077806    -0.086528    -0.096661    -0.090781    -0.070635   
2     -0.077193    -0.096045    -0.100846    -0.080868    -0.058268   
3     -0.086073    -0.109648    -0.107964    -0.080322    -0.061459   
4     -0.113837    -0.135299    -0.133049    -0.097750    -0.076387   
..          ...          ...          ...          ...          ...   
64    -0.017255    -0.023807    -0.025319    -0.000999    -0.000898   
65    -0.002743    -0.008813    -0.004157     0.019904     0.005379   
66     0.027916     0.042996     0.044870     0.049920     0.017446   
67     0.015685     0.027851     0.027902     0.029382     0.005115   
68    -0.001519     0.006195     0.001917    -0.005573    -0.027112   

    outvvel_700  outvvel_850  outvvel_925  outvvel_975  outvvel_1000  
0     -0.090385    -0.059009    -0.037655    -0.016894     -0.007100  
1     -0.053375    -0.028313    -0.013505     0.002304      0.008856  
2     -0.034209    -0.011050     0.006537     0.019232      0.025641  
3     -0.047983    -0.029134    -0.014392    -0.010640     -0.008889  
4     -0.067142    -0.048591    -0.034726    -0.033828     -0.035239  
..          ...          ...          ...          ...           ...  
64     0.007622     0.008740    -0.001515    -0.000483      0.001987  
65    -0.000754    -0.007770    -0.006062     0.001463      0.004688  
66     0.008556     0.009051     0.014194     0.024705      0.029293  
67     0.002133     0.008836     0.011722     0.016127      0.020081  
68    -0.032866    -0.025765    -0.023448    -0.028676     -0.027834  

[69 rows x 300 columns]

In [200]:
df1=ts_lehar
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

Pmin     w10max        div100    div200        div250  \
0   1008.354375  12.089149 -1.114373e-06  0.000028  4.712652e-05   
1   1006.899375  12.099492  2.207192e-05  0.000028  4.880802e-05   
2   1008.137500  12.946914  3.799148e-05  0.000048  5.157499e-05   
3   1009.246250  13.524454  6.559324e-05  0.000044  4.015736e-05   
4   1007.101250  11.577554  6.630583e-05  0.000039  3.377555e-05   
..          ...        ...           ...       ...           ...   
64  1004.356875  15.110038  2.514235e-05 -0.000003 -1.469511e-05   
65  1003.528125  13.975189  1.180500e-05 -0.000001 -8.951417e-06   
66  1004.117500  13.896652 -1.453482e-07  0.000016  9.460861e-06   
67  1006.631250  13.558363  4.308552e-06  0.000013  6.167821e-06   
68  1006.995000  13.248140  1.028076e-05  0.000005 -6.598098e-07   

          div300        div400        div500     eqt1000      eqt200  ...  \
0   2.683366e-05  5.358663e-06 -3.709186e-06  321.301117  347.093323  ...   
1   3.506813e-05  3.678299e-06 -7.396683e-06  328.678925  347.569061  ...   
2   4.190413e-05 -3.900933e-06 -1.667449e-07  330.235901  349.061005  ...   
3   2.956255e-05 -7.985167e-06  9.848300e-07  330.619781  349.082031  ...   
4   1.697406e-05 -1.857898e-07 -9.271409e-07  329.841675  349.316895  ...   
..           ...           ...           ...         ...         ...  ...   
64 -3.176734e-06  1.943779e-06  8.115904e-09  328.671539  348.846008  ...   
65  1.838834e-06  6.610712e-06  1.255684e-06  327.732666  348.234741  ...   
66  3.836846e-06  5.894540e-07 -8.441037e-06  322.099609  348.801941  ...   
67  4.318058e-06  4.497268e-06 -5.560843e-06  322.856384  348.744019  ...   
68  1.322514e-07  6.795401e-06  2.703427e-06  327.365234  348.565765  ...   

    outvvel_250  outvvel_300  outvvel_400  outvvel_500  outvvel_600  \
0     -0.075850    -0.080951    -0.104787    -0.111518    -0.102335   
1     -0.077806    -0.086528    -0.096661    -0.090781    -0.070635   
2     -0.077193    -0.096045    -0.100846    -0.080868    -0.058268   
3     -0.086073    -0.109648    -0.107964    -0.080322    -0.061459   
4     -0.113837    -0.135299    -0.133049    -0.097750    -0.076387   
..          ...          ...          ...          ...          ...   
64    -0.017255    -0.023807    -0.025319    -0.000999    -0.000898   
65    -0.002743    -0.008813    -0.004157     0.019904     0.005379   
66     0.027916     0.042996     0.044870     0.049920     0.017446   
67     0.015685     0.027851     0.027902     0.029382     0.005115   
68    -0.001519     0.006195     0.001917    -0.005573    -0.027112   

    outvvel_700  outvvel_850  outvvel_925  outvvel_975  outvvel_1000  
0     -0.090385    -0.059009    -0.037655    -0.016894     -0.007100  
1     -0.053375    -0.028313    -0.013505     0.002304      0.008856  
2     -0.034209    -0.011050     0.006537     0.019232      0.025641  
3     -0.047983    -0.029134    -0.014392    -0.010640     -0.008889  
4     -0.067142    -0.048591    -0.034726    -0.033828     -0.035239  
..          ...          ...          ...          ...           ...  
64     0.007622     0.008740    -0.001515    -0.000483      0.001987  
65    -0.000754    -0.007770    -0.006062     0.001463      0.004688  
66     0.008556     0.009051     0.014194     0.024705      0.029293  
67     0.002133     0.008836     0.011722     0.016127      0.020081  
68    -0.032866    -0.025765    -0.023448    -0.028676     -0.027834  

[69 rows x 270 columns]

In [201]:
df1.to_csv(output+'timeseries_all_nio_lehar.csv')

In [202]:
mainmadi=pd.read_csv(output+'inner/inner_nio_all_madi.csv',delimiter=r",")
mainmadi.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainmadi=mainmadi.drop('a', axis=1)

pdf_madi=pd.read_csv(target+'2013_pmin_nio_madi.csv',delimiter=r",")
pdf_madi.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_madi=pdf_madi.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2013_nio_madi.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_madi=pd.DataFrame.from_dict(TCs_ts[3])
rlev_madi=pd.DataFrame.from_dict(TCr_ts[3])
prlev_madi=pd.DataFrame.from_dict(TCpr_ts[3])
prlev_madi['outmslp']=prlev_madi['outmslp']/100

ts1_madi=pd.concat([mainmadi,extra,slev_madi,rlev_madi,prlev_madi], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[3]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[3]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[3]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[3]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_madi=pd.DataFrame.from_dict(alldict1)

ts_madi=pd.concat([pdf_madi,ts1_madi,plev_madi], axis=1, join='inner')
ts_madi

Pmin     w10max        div100    div200    div250        div300  \
0   1004.133125  12.804338  5.682303e-05  0.000029  0.000005  2.855302e-06   
1   1005.436875  14.013905  4.920553e-05  0.000027  0.000015  1.143469e-05   
2   1003.482500  14.716041  5.068671e-05  0.000043  0.000033  2.169186e-05   
3    999.986875  16.741007  5.496447e-05  0.000032  0.000020  1.091589e-05   
4   1000.411250  15.737922  4.695368e-05  0.000031  0.000010 -3.135810e-07   
..          ...        ...           ...       ...       ...           ...   
56   999.691875  15.974120  7.700743e-06  0.000008  0.000005 -6.896110e-06   
57  1002.498125  14.691542 -5.833215e-06  0.000006  0.000011 -8.460438e-06   
58  1002.770625  13.998476 -6.777248e-06  0.000010  0.000004 -4.489216e-06   
59  1001.520000  13.904409  3.494342e-07  0.000006  0.000003 -3.449458e-07   
60  1003.021875  10.787711  1.732439e-05  0.000007  0.000007  2.600036e-06   

          div400    div500     eqt1000      eqt200  ...  outvvel_250  \
0   9.056144e-06  0.000005  334.250183  350.075317  ...    -0.102823   
1   9.177023e-06  0.000004  335.062286  350.029358  ...    -0.107940   
2   9.461563e-06  0.000009  334.558441  351.447937  ...    -0.113464   
3  -1.053294e-06 -0.000002  335.559875  351.955933  ...    -0.136502   
4  -3.554412e-06 -0.000007  334.122925  351.462250  ...    -0.151632   
..           ...       ...         ...         ...  ...          ...   
56 -3.916429e-06  0.000002  324.595642  349.427551  ...     0.031673   
57 -2.476413e-07 -0.000002  319.606506  349.575317  ...     0.035156   
58 -1.563069e-07  0.000003  314.339722  349.760559  ...     0.036211   
59 -5.560501e-06  0.000006  317.527100  349.655884  ...     0.042284   
60 -3.554255e-06  0.000008  325.036682  350.816406  ...     0.047623   

    outvvel_300  outvvel_400  outvvel_500  outvvel_600  outvvel_700  \
0     -0.113713    -0.112274    -0.113550    -0.099597    -0.072988   
1     -0.116796    -0.118408    -0.120267    -0.106681    -0.079686   
2     -0.122739    -0.124034    -0.125457    -0.116808    -0.098970   
3     -0.146857    -0.148642    -0.143541    -0.137158    -0.123043   
4     -0.164470    -0.152102    -0.121773    -0.089759    -0.064388   
..          ...          ...          ...          ...          ...   
56     0.029762     0.051239     0.055527     0.057883     0.074211   
57     0.032319     0.045568     0.047117     0.040800     0.052576   
58     0.026509     0.026513     0.016189    -0.002175     0.000071   
59     0.027987     0.017619     0.013089     0.003613    -0.000579   
60     0.034944     0.023161     0.019395     0.011736     0.007403   

    outvvel_850  outvvel_925  outvvel_975  outvvel_1000  
0     -0.035423    -0.017891     0.003958      0.014975  
1     -0.044035    -0.027623    -0.004294      0.007705  
2     -0.078158    -0.065081    -0.046881     -0.036839  
3     -0.080377    -0.055858    -0.032895     -0.021349  
4     -0.030347    -0.017055    -0.003620      0.003706  
..          ...          ...          ...           ...  
56     0.038734     0.026511     0.032147      0.033014  
57     0.034093     0.028961     0.026521      0.024427  
58    -0.012452    -0.013170    -0.020375     -0.023542  
59    -0.015331    -0.013729    -0.014712     -0.017152  
60    -0.000799     0.005665     0.008762      0.008597  

[61 rows x 300 columns]

In [203]:
df1=ts_madi
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

Pmin     w10max        div100    div200    div250        div300  \
0   1004.133125  12.804338  5.682303e-05  0.000029  0.000005  2.855302e-06   
1   1005.436875  14.013905  4.920553e-05  0.000027  0.000015  1.143469e-05   
2   1003.482500  14.716041  5.068671e-05  0.000043  0.000033  2.169186e-05   
3    999.986875  16.741007  5.496447e-05  0.000032  0.000020  1.091589e-05   
4   1000.411250  15.737922  4.695368e-05  0.000031  0.000010 -3.135810e-07   
..          ...        ...           ...       ...       ...           ...   
56   999.691875  15.974120  7.700743e-06  0.000008  0.000005 -6.896110e-06   
57  1002.498125  14.691542 -5.833215e-06  0.000006  0.000011 -8.460438e-06   
58  1002.770625  13.998476 -6.777248e-06  0.000010  0.000004 -4.489216e-06   
59  1001.520000  13.904409  3.494342e-07  0.000006  0.000003 -3.449458e-07   
60  1003.021875  10.787711  1.732439e-05  0.000007  0.000007  2.600036e-06   

          div400    div500     eqt1000      eqt200  ...  outvvel_250  \
0   9.056144e-06  0.000005  334.250183  350.075317  ...    -0.102823   
1   9.177023e-06  0.000004  335.062286  350.029358  ...    -0.107940   
2   9.461563e-06  0.000009  334.558441  351.447937  ...    -0.113464   
3  -1.053294e-06 -0.000002  335.559875  351.955933  ...    -0.136502   
4  -3.554412e-06 -0.000007  334.122925  351.462250  ...    -0.151632   
..           ...       ...         ...         ...  ...          ...   
56 -3.916429e-06  0.000002  324.595642  349.427551  ...     0.031673   
57 -2.476413e-07 -0.000002  319.606506  349.575317  ...     0.035156   
58 -1.563069e-07  0.000003  314.339722  349.760559  ...     0.036211   
59 -5.560501e-06  0.000006  317.527100  349.655884  ...     0.042284   
60 -3.554255e-06  0.000008  325.036682  350.816406  ...     0.047623   

    outvvel_300  outvvel_400  outvvel_500  outvvel_600  outvvel_700  \
0     -0.113713    -0.112274    -0.113550    -0.099597    -0.072988   
1     -0.116796    -0.118408    -0.120267    -0.106681    -0.079686   
2     -0.122739    -0.124034    -0.125457    -0.116808    -0.098970   
3     -0.146857    -0.148642    -0.143541    -0.137158    -0.123043   
4     -0.164470    -0.152102    -0.121773    -0.089759    -0.064388   
..          ...          ...          ...          ...          ...   
56     0.029762     0.051239     0.055527     0.057883     0.074211   
57     0.032319     0.045568     0.047117     0.040800     0.052576   
58     0.026509     0.026513     0.016189    -0.002175     0.000071   
59     0.027987     0.017619     0.013089     0.003613    -0.000579   
60     0.034944     0.023161     0.019395     0.011736     0.007403   

    outvvel_850  outvvel_925  outvvel_975  outvvel_1000  
0     -0.035423    -0.017891     0.003958      0.014975  
1     -0.044035    -0.027623    -0.004294      0.007705  
2     -0.078158    -0.065081    -0.046881     -0.036839  
3     -0.080377    -0.055858    -0.032895     -0.021349  
4     -0.030347    -0.017055    -0.003620      0.003706  
..          ...          ...          ...           ...  
56     0.038734     0.026511     0.032147      0.033014  
57     0.034093     0.028961     0.026521      0.024427  
58    -0.012452    -0.013170    -0.020375     -0.023542  
59    -0.015331    -0.013729    -0.014712     -0.017152  
60    -0.000799     0.005665     0.008762      0.008597  

[61 rows x 270 columns]

In [204]:
df1.to_csv(output+'timeseries_all_nio_madi.csv')